In [17]:
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
import textdistance

`extract_unique_locations`

Het corpusbestand `corpus_1568-1609.ndjson`, genoemd `corpus`, wordt geopend. De informatie onder `tekst` wordt gelezen als html-code, waarin alle `locatie`-tags worden verzameld. Alle unieke tags worden toegevoegd aan de set `unique_variations.`

In [18]:
def extract_unique_locations(corpus):
    unique_variations = set()

    with open(corpus, 'r', encoding='utf-8') as corpus_file:
        for line in corpus_file:
            data = json.loads(line)
            soup = bs(data['tekst'], 'html.parser')
            locatie_tags = soup.find_all('locatie')
            variation_values = [tag.text.strip() for tag in locatie_tags]
            unique_variations.update(variation_values)

    return list(unique_variations)

`import_geonames_data`

De GeoNames-tabel wordt geïmporteerd. De waardes voor `name` en `alternatenames` worden in kleine letters geschreven om te voorkomen dat er verschillen worden opgemerkt op basis van kleine letters en hoofdletters.

De informatie over de stadsnaam en de (spellings)variaties van die stadsnaam worden opgeslagen in een `dict` genaamd `geonames_dict`. 

In [19]:
def import_geonames_data(geonames_data):
    geonames_dataframe = pd.read_csv(geonames_data)
    geonames_dict = {}
    
    for index, row in geonames_dataframe.iterrows():
        name = row['name'].lower()
        alternatenames = row['alternatenames'].lower()
        alternatenames_list = [name.strip() for name in alternatenames.strip("[]").replace("'", "").split(",")]
        geonames_dict[name] = alternatenames_list

    return geonames_dict

`entity_linking_locations`

De locatietags (`corpus_locations`) worden vergeleken met de data uit GeoNames (`geonames_data`). Als een locatie één op één overeenkomt met een variatie uit GeoNames, is het resultaat van het entity linking proces met de waarde uit GeoNames die bij de variatie past. Als er geen overeenkomstige waarden zijn, wordt op basis van de afstanden tussen waardes uit het corpus en waardes uit de GeoNames-data gekeken welke waarden het dichtst bij elkaar liggen. Als er een waarde is met een afstand groter dan 0.7, wordt variatie met de hoogste score toegeschreven als resultaat.

In [20]:
def entity_linking_locations(corpus_locations, geonames_data):
    result_dict = {}

    for variation in corpus_locations:
        variation_lower = variation.lower()

        if variation_lower in geonames_data:
            result_dict[variation] = [variation_lower]

        else:
            best_match_city = ''
            max_similarity = 0

            for city, variants in geonames_data.items():
                for variant in variants:
                    similarity_score = textdistance.damerau_levenshtein.normalized_similarity(variation_lower, variant)
                
                    if similarity_score > max_similarity:
                        max_similarity = similarity_score
                        best_match_city = city

            if max_similarity >= 0.7:
                result_dict[variation] = [best_match_city]
            else:
                result_dict[variation] = []

    return result_dict

`main`

Entity-linkingproces:
1. De GeoNames-tabel wordt geladen;
2. De unieke locatietags in het corpus worden verzameld;
3. De unieke locatietags in het corpus worden gekoppeld aan de GeoNames-data.

De resultaten worden geëxporteerd naar `entity linking_results.csv`

In [21]:
def main():
    geonames_data = 'data/geonames_data.csv'
    corpus_file_path = 'data/corpus_1568-1609.ndjson'

    geonames_dict = import_geonames_data(geonames_data)
    location_list = extract_unique_locations(corpus_file_path)
    result_dict = entity_linking_locations(location_list, geonames_dict)

    dataframe = pd.DataFrame(result_dict.items())
    dataframe.to_csv('data/entity linking_results.csv', index=False)

In [22]:
if __name__ == '__main__':
    main()